<a href="https://colab.research.google.com/github/justintran12/CS598-DLH-FinalProject/blob/main/DL4H_Team_40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Notebook to Google Drive

Note: not needed anymore, use gdown instead to download the public file from google drive

Upload the data, pretrianed model, figures, etc to your Google Drive, then mount this notebook to Google Drive. After that, you can access the resources freely.

Instruction: https://colab.research.google.com/notebooks/io.ipynb

Example: https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q

Video: https://www.youtube.com/watch?v=zc8g8lGcwQU

In [ ]:
# not needed anymore, use gdown instead to download the public file from google drive
'''
from google.colab import drive
drive.mount('/content/drive')
'''

# Video Link

https://drive.google.com/file/d/1fnz2Jv7rojMCsDmE_PFbaQQ5xiZ_2PEQ/view?usp=drive_link

# Introduction

  The original paper addresses the problem of cancer type detection through inferring cancer marker genes. By solving this problem, cancer will be more easily detected in the early stages, leading to drastically improved survivability rates. The challenge is that it is difficult to take the tissue of origin into account when developing models, due to loss of accuracy when doing so. Also, it is time and resource intensive to train a model with many layers that is able to detect cancer marker genes accurately.

  Current state of the art methods include using a fully connected deep neural network (Ahn, et al), using a k-nearest neighbor alrgorithm with a genetic algorithm for gene selection (Li et al), convolutional neural network (CNN) model with a 2D mapping of the gene expression samples as input matrices (Lyu et al). The KNN model achieved greater than 90% accuracy for predicting 31 cancer types. The CNN model achieved greater than 95% accuracy for 33 The Cancer Genome Atlas (TCGA) cancer types.

  The original paper proposed 3 different CNN models, a 1D-CNN model, 2D-Vanilla-CNN model, and a 2D-Hybrid-CNN model. Each model uses just one convolution layer.
  
  The 1D-CNN model takes a 1-D vector as input and applies 1-D convolution kernels. The stride of the convolution and the kernel size are the same in order to capture only the global featrues in the data, since the authors were not sure that there were any correlations between neighboring gene expressions.

  The 2D-Vanilla-CNN takes in a 2-D matrix and is similar to commonly used CNNs that take in image data and attempt to capture local features. Since a 2-D matrix is an input, the gene expression data is reshpaed into a 100 by 71 matrix.

  The 2D-Hybrid-CNN model takes in a 2-D matrix as input, but uses 1-D convolution kernels. One kernel slides horizontally over each column and the other slides vertically over each row. This hybrid model has the positive benefits from both the 1D-CNN and the 2D-Vanilla, it can take in 2-D inputs, while using simple 1-D kernels. The original authors also believed this model would capture more global unstructured features.

  For each model, ReLU is used as the activation function, the input is first fed into a convolution layer, then a pooling layer, then a fully connected layer, then a softmax prediction layer to generate the output. For the Hybrid model, the outputs of the pooling layer are concatenated together before being fed into the fully connected layer.

  Categorical Cross Entropy is the loss function, Categorical accuracy is training metric and the Adam optimizer were selected for all 3 CNN models

  These 3 models proposed by this paper, are able to achieve similar accuracy metrics compared to a more complicated multi-layer model while using less resources while training and tuning hyperparameters. Having less layers also can prevent overfitting, especially in this case, as there are limited samples relative to the number of parameters.

  The 1D-CNN and 2D-Hybrid-CNN achieved comparable accuracy (95.7%), which improves the result (95.6%) of the 2D-3Layer-CNN. Also, the 1D-CNN model uses significantly less hyperparameters (200 thousand) compared to the 2D-3Layer-CNN (26 million).

  The contribution of this paper is the three lighter models that have comparable accuracy metrics to the more traditional multi-layer models. The lighter models use less hyperparameters, take less time and resources to train, and are less prone to overfitting. Also when taking tissue of origin into account, the three proposed models have accetable accuracy results.


# Scope of Reproducibility:

1.   Hypothesis 1: Proposed single convolution layer CNN models can achieve similar or better accuracy metrics compared to traditional multi-layer CNN models.

  Experiment: All three CNN models were trained with all 10,340 tumor samples initially. The loss function values over 50 epochs and categorical accuracy were measured. I will use 1 epoch in this reproduction as an example of the training, as multiple epochs takes too long to finish training. An 80-20% train-validation split was used.

2.   Hypothesis 2: Proposed single convolution layer CNN models can achieve acceptable accuracy metrics when taking tissue of origin into account.

  Experiment: Similar to experiment for hypothesis 1 except a new label is introduced in the prediction layer which takes all normal samples regardless of their original tissue type designation. This 34th node removes the trace of tissue of origins from cancer samples.



# Methodology

## Environment

Use Python 3.

sklearn version 1.2.2, keras packages needed. Do not install scikeras.

In [ ]:
# run the following if needed
#pip install keras
#pip install sklearn
# DO NOT INSTALL SCIKERAS
#pip install scikeras

# sklearn version should be 1.2.2
#import sklearn; print(sklearn.__version__)

In [ ]:
# import packages and dependencies
import gdown
import numpy as np
import time
from google.colab import drive

import pickle
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import collections
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, Input, LeakyReLU, BatchNormalization, concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import precision_recall_curve, roc_curve, auc, average_precision_score
from sklearn.model_selection import StratifiedKFold
from collections import Counter
#from scikeras.wrappers import KerasClassifier
#import keras

##  Data

  The data is real-world, pan-cancer RNA-Seq data taken from The Cancer Genome Atlas through an R package called TCGABiolinks in 2018. There is no direct link to the raw data in the paper. The dataset contains 10340 samples for 33 cancer types and 714 samples for 23 normal tissues. The paper pre-processed the data and stored the post-processed data in pickle files for the 33 class data and 34 class data (33 classes plus normal). The pickle files are available for download from the author's github under the README file (Mostavi) [2], not the chenlabgccr github. For my reproduction, I downloaded the pickle files and stored them in the same directory as this notebook in Google Drives.

  In the data pre-process step, genes with low information burden (mean < 0.5 and standard deviation < 0.8) were removed across all samples. This was done to remove noise-sensitive features from the dataset. 7091 genes remained after filtering in this pre-process step. Afterwards, nine zeros were added to the dataset to round the input dimension to 7100 to allow for easier modeling of the data. I added notes in the below code to show how this was done. The indices of the samples to use were already included in the pickle file by the author.

  The code to load the data for the 33 class case and the 34 class case is slightly different, so I just separated them into two functions for easier use.

  The cross validation split is 80-20% for training and validation.

In [ ]:
# Loading the data for 33 class and 34 class
def load_processed_data_33_Class(processed_data_dir_1, processed_data_dir_2):

  ## load the pickle files
  A = open(processed_data_dir_2, 'rb')
  [dropped_genes_final, dropped_gene_name, dropped_Ens_id, samp_id_new, diag_name_new,
  project_ids_new] = pd.read_pickle(A)
  A.close()

  f = open(processed_data_dir_1 , 'rb')
  [_, _, _, _, remain_cancer_ids_ind, remain_normal_ids_ind] = pd.read_pickle(f)
  f.close()

  ## embedding labels
  # integer encode
  label_encoder = LabelEncoder()
  integer_encoded = label_encoder.fit_transform(project_ids_new)
  # binary encode
  onehot_encoder = OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

  ## filter out the dropped genes
  X_cancer_samples =dropped_genes_final.iloc[:,remain_cancer_ids_ind].T.values
  X_normal_samples = dropped_genes_final.iloc[:,remain_normal_ids_ind].T.values
  onehot_encoded_cancer_samples = onehot_encoded[remain_cancer_ids_ind]
  onehot_encoded_normal_samples = onehot_encoded[remain_normal_ids_ind]

  ## add nine zeros to the end of each sample
  X_cancer_samples_mat = np.concatenate((X_cancer_samples,np.zeros((len(X_cancer_samples),9))),axis=1)
  X_cancer_samples_mat = np.reshape(X_cancer_samples_mat, (-1, 71, 100))

  input_Xs = X_cancer_samples_mat
  y_s = project_ids_new[remain_cancer_ids_ind]

  ## This line is useful when only one fold training is needed
  x_train, x_test, y_train, y_test = train_test_split(X_cancer_samples_mat, onehot_encoded_cancer_samples,
                                                      stratify= onehot_encoded_cancer_samples,
                                                      test_size=0.25, random_state=42)

  img_rows, img_cols = len(x_test[0]), len(x_test[0][0])

  # hybrid model flips the rows and cols
  img_rows_hyb, img_cols_hyb = len(x_test[0][0]), len(x_test[0])
  num_classes = len(y_train[0])

  return input_Xs, y_s, img_rows, img_cols, num_classes, img_rows_hyb, img_cols_hyb, x_train, x_test, y_train, y_test

def load_processed_data_34_Class(processed_data_dir_1, processed_data_dir_2):
  A = open(processed_data_dir_2, 'rb')
  [dropped_genes_final, dropped_gene_name, dropped_Ens_id, samp_id_new, diag_name_new,
  project_ids_new] = pd.read_pickle(A)
  A.close()

  f = open(processed_data_dir_1, 'rb')
  [_, _, _, _, remain_cancer_ids_ind, remain_normal_ids_ind] = pd.read_pickle(f)
  f.close()

  X_cancer_samples = dropped_genes_final.iloc[:,remain_cancer_ids_ind].T.values
  X_normal_samples = dropped_genes_final.iloc[:,remain_normal_ids_ind].T.values

  name_cancer_samples = project_ids_new[remain_cancer_ids_ind]
  name_normal_samples = ['Normal Samples'] *len(X_normal_samples)

  X_cancer_samples_34 = np.concatenate((X_cancer_samples,X_normal_samples))
  X_names = np.concatenate((name_cancer_samples,name_normal_samples))
  X_cancer_samples_mat = np.concatenate((X_cancer_samples_34,np.zeros((len(X_cancer_samples_34),9))),axis=1)
  X_cancer_samples_mat = np.reshape(X_cancer_samples_mat, (-1, 71, 100))

  input_Xs = X_cancer_samples_mat
  y_s = X_names

  img_rows, img_cols = len(input_Xs[0][0]), len(input_Xs[0])
  num_classes = len(set(y_s))

  return input_Xs, X_cancer_samples_34, y_s, img_rows, img_cols, num_classes

def load_processed_data_HP_Tuning(processed_data_dir_1, processed_data_dir_2):
  # X_cancer_samples_mat same from load_processed_data_33_Class
  pass


In [ ]:
# getting the preprocessed data via gdown
url_1 = 'https://drive.google.com/file/d/1RVdIFcMYJyd1_PyZLO0HUBnmprPQlVUK/view?usp=sharing'
url_2 = 'https://drive.google.com/file/d/1tFlZU7hDIvB_6daxc1UPPj-7j8MMq1Z3/view?usp=sharing'
output_1 = 'TCGA_new_pre_first.pckl'
output_2 = 'TCGA_new_pre_second.pckl'
gdown.download(url=url_1, output=output_1, fuzzy=True)
gdown.download(url=url_2, output=output_2, fuzzy=True)
input_Xs_33, y_s_33, img_rows_33, img_cols_33, num_classes_33, img_rows_hyb_33, img_cols_hyb_33, x_train_HP, x_test_HP, y_train_HP, y_test_HP = load_processed_data_33_Class(output_1, output_2)
input_Xs_34, X_cancer_samples_34, y_s_34, img_rows_34, img_cols_34, num_classes_34 = load_processed_data_34_Class(output_1, output_2)

'''
# getting the data via google drive mounting method
processed_data_dir_1 = '/content/drive/My Drive/Colab Notebooks/TCGA_new_pre_first.pckl'
processed_data_dir_2 = '/content/drive/My Drive/Colab Notebooks/TCGA_new_pre_second.pckl'
input_Xs_33, y_s_33, img_rows_33, img_cols_33, num_classes_33, img_rows_hyb_33, img_cols_hyb_33, x_train_HP, x_test_HP, y_train_HP, y_test_HP = load_processed_data_33_Class(processed_data_dir_1, processed_data_dir_2)
input_Xs_34, X_cancer_samples_34, y_s_34, img_rows_34, img_cols_34, num_classes_34 = load_processed_data_34_Class(processed_data_dir_1, processed_data_dir_2)
'''

##   Model

  Original Paper Citation: See [1] in References section.

  Original Paper GitHub Repo: https://github.com/chenlabgccri/CancerTypePrediction


  The model architecture parameters were found from hyperparameter training tuned via Grid Search method. The dense layer size, number and size of kernels, and the stride of kernels were tuned using this method. For the 1D-CNN model, the stride was set to the same length as the kernel size in order to only capture global features. All three models proposed in the paper only use 1 convolution layer.

  The 1D-CNN takes in a 1D vector representing gene expressions as input and applies 1D convolution kernels to the input. The output of the convolution layer is input to a max pooling layer, then a fully-connected (FC) layer with ReLU as the activation function, then the prediction layer which uses softmax as the activation function.

  The 2D-Vanilla-CNN takes in a 2D matrix input similar to an image. The input gene expression was reshaped into 2D space before being fed into the model. The 2D CNN includes the convolutional layer with the 2D kernel, a maxpooling layer, a FC layer, and a prediction layer with similar activation functions as the 1D-CNN model. In this model, the kernel stride was tuned from the hyperparameter training in order to best capture local features.

  The Hybrid-CNN takes in a 2D matrix input similar to the Vanilla-CNN, but uses two 1D kernels that slide across the rows and columns of the input separately. The outputs of the two 1D kernels are then passed through a maxpooling layer, then concatenated and fed into the FC and prediction layers with similar activation functions as the 1D-CNN model. The authors of the original paper believed this design can capture more global unstructured features in the input gene expression.

  For every model, categorical_crossentropy loss function, and the adam optimzer were used. Categorical_accuracy was used as the main training metric used.

  The original paper's github replicated the model code in seperate files for each model. I just made each model a class for re-usability. The only issue is that the 2D-Vanilla model is slightly different (stride used is different) for the 33 class than the 34 class, so I had two separate model classes for the 2D-Vanilla 33 class case and the 34 class case.


In [ ]:
# Defining models
class One_CNN():
  # use this class to define your model
  def __init__(self, input_shape, num_classes):
    self.model = Sequential()
    ## *********** First layer Conv
    self.model.add(Conv2D(32, kernel_size=(1, 71), strides=(1, 1),
              input_shape=input_shape))
    self.model.add(Activation('relu'))
    self.model.add(MaxPooling2D(1, 2))
    ## ********* Classification layer
    self.model.add(Flatten())
    self.model.add(Dense(128, activation='relu'))
    self.model.add(Dense(num_classes, activation='softmax'))
    self.model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['categorical_accuracy'])

class Vanilla_33():
  # use this class to define your model
  def __init__(self, input_shape, num_classes):
    self.model = Sequential()
    ## *********** First layer Conv
    self.model.add(Conv2D(32, kernel_size=(10, 10), strides=(1, 1),
              input_shape=input_shape))
    self.model.add(Activation('relu'))
    self.model.add(MaxPooling2D(2, 2))
    ## ********* Classification layer
    self.model.add(Flatten())
    self.model.add(Dense(128, activation='relu'))
    self.model.add(Dense(num_classes, activation='softmax'))
    self.model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['categorical_accuracy'])

class Hybrid():
  # use this class to define your model
  def __init__(self, input_img, num_classes):
    tower_1 = Conv2D(32, (1, 71), activation='relu')(input_img)
    tower_1 = MaxPooling2D(1, 2)(tower_1)
    tower_1 = Flatten()(tower_1)

    tower_2 = Conv2D(32, (100, 1), activation='relu')(input_img)
    tower_2 = MaxPooling2D(1, 2)(tower_2)
    tower_2 = Flatten()(tower_2)

    output = concatenate([tower_1, tower_2], axis=1)
    out1 = Dense(128, activation='relu')(output)
    last_layer = Dense(num_classes, activation='softmax')(out1)
    # typo with original code, should be inputs and outputs not input and output
    self.model = Model(inputs=[input_img], outputs=last_layer)
    self.model.output_shape

    self.model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['categorical_accuracy'])

class Vanilla_34():
  # use this class to define your model
  def __init__(self, input_shape, num_classes):
    self.model = Sequential()
    ## *********** First layer Conv
    self.model.add(Conv2D(32, kernel_size=(10, 10), strides=(2, 2),
              input_shape=input_shape))
    self.model.add(Activation('relu'))
    self.model.add(MaxPooling2D(2, 2))
    ## ********* Classification layer
    self.model.add(Flatten())
    self.model.add(Dense(128, activation='relu'))
    self.model.add(Dense(num_classes, activation='softmax'))
    self.model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['categorical_accuracy'])

def HP_Tuning_model_1D(dense_layer_sizes, filters, kernel_size):
  x_train = x_train_HP.reshape(x_train_HP.shape[0], img_rows_33, img_cols_33, 1)
  x_test = x_test_HP.reshape(x_test_HP.shape[0], img_rows_33, img_cols_33, 1)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')

  img_rows, img_cols = len(x_test[0]), len(x_test[0][0])
  num_classes = len(y_train_HP[0])
  input_shape = (img_rows, img_cols, 1)

  model = Sequential()
  ## *********** First layer Conv
  model.add(Conv2D(filters, kernel_size=kernel_size, strides=(1, 1),
                     input_shape=input_shape))
  # model.add(BatchNormalization())
  model.add(Activation('relu'))
  # model.add(LeakyReLU())
  model.add(MaxPooling2D(1, 2))
  model.output_shape

  ## ********* Classification layer
  model.add(Flatten())
  model.add(Dense(dense_layer_sizes, activation='relu'))

  model.add(Dense(num_classes, activation='softmax'))
  #model.output_shape

  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['categorical_accuracy'])
  #model.summary()
  return model

def HP_Tuning_model_Vanilla(dense_layer_sizes, filters, stride, kernel_size):
  x_train = x_train_HP.reshape(x_train_HP.shape[0], img_rows_33, img_cols_33, 1)
  x_test = x_test_HP.reshape(x_test_HP.shape[0], img_rows_33, img_cols_33, 1)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')

  img_rows, img_cols = len(x_test[0]), len(x_test[0][0])
  num_classes = len(y_train_HP[0])
  input_shape = (img_rows, img_cols, 1)

  model = Sequential()
  ## *********** First layer Conv
  model.add(Conv2D(filters, kernel_size=kernel_size, strides=stride,
                    input_shape=input_shape))
  # model.add(BatchNormalization())
  model.add(Activation('relu'))
  # model.add(LeakyReLU())
  model.add(MaxPooling2D(2, 2))
  model.output_shape

  ## ********* Classification layer
  model.add(Flatten())
  model.add(Dense(dense_layer_sizes, activation='relu'))

  model.add(Dense(num_classes, activation='softmax'))
  model.output_shape

  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['categorical_accuracy'])
  model.summary()
  return model

## Training and Evaluation

For training with 33 classes, the 1D-CNN requires 2218529 parameters to be tuned during training, 2D-Vanilla-CNN requires 5721537 parameters to be tuned, Hybrid-CNN requires 362177 parameters to be tuned.

For training with 33 classes plus the normal class, the 1D-CNN requires 211618 parameters to be tuned during training, 2D-Vanilla-CNN requires 1420866 parameters to be tuned, Hybrid-CNN requires 362306 parameters to be tuned. The normal class is added to take the impact of tissue of origin into account. A new label was introduced in the prediction layer, where it takes all normal samples (regardless of their original tissue type designation). The overall accuracy of the models with the introduction of the normal class is sligtly lower due to the presence of the normal samples.

For evaluation categorical accuracy was used as the metric for comparing models.

## Hyperparameters

The hyperparameters were tuned using Grid Search method. The hyperparameters tuned were the dense layer size, number and size of kernels, and the stride of kernels.

The hyperparameter tuning code provided in the original paper's github is incompatible due to the removal of KerasClassifier from the sklearn package. To use KerasClassifier, you need to install the latest version of sklearn and scikeras, but this breaks the code when using OneHotEncoder when preprocessing the data.

The original paper uses 25 epochs, so this would take a very long time to run anyways, so it is not practical to actually tune the hyperparameters in this environment and not needed for this paper reproduction. I will just use the hyperparamters given by the original paper after they ran the hyperparameter tuning code.


In [ ]:
# This will not run because it requires KerasClassifier from latest version of sklearn, which breaks the OneHotEncoder.
'''
dense_size_candidates = [64, 128, 512]
my_classifier = KerasClassifier(HP_Tuning_model_1D, batch_size=128)
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [25],
                                     'filters': [8,16, 32, 64],
                                     'kernel_size': [(1, 71)]},
                         scoring='neg_log_loss',
                         n_jobs=1)
validator.fit(x_train, y_train)

print('The parameters of the best model for 1D-CNN 33 Class are: ')
print(validator.best_params_)

dense_size_candidates = [64, 128, 512]
my_classifier = KerasClassifier(HP_Tuning_model_Vanilla, batch_size=128)
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [25],
                                     'filters': [8, 16, 32, 64], 'stride': [(1, 1),(2, 2),(5, 5)],
                                     'kernel_size': [(7, 7), (10, 10), (15, 15), (20, 20)]},
                         scoring='neg_log_loss',
                         n_jobs=1)
validator.fit(x_train, y_train)

print('The parameters of the best model for 2D-Vanilla-CNN 33 Class are: ')
print(validator.best_params_)
'''

## Computational Requirements

For my reproduction, the total runtime for the 33 class case is 724 s and 181 s for the 34 class case. For the 33 class case, average runtime for the 1D-CNN is 33 s, average runtime for the 2D-Vanilla is 101 s, average runtime for the Hybrid-CNN is 8 s. For the 34 class case, average runtime for the 1D-CNN is 4 s, average runtime for the 2D-Vanilla is 23 s, average runtime for the Hybrid-CNN is 7 s. The number of training epochs is 1, total number of trials is 1, type of hardware used is the Python 3 Google Compute Engine provided by Google Colab.

In the original paper the total runtime is 80.3 seconds for the 1D-CNN, 94 seconds for the 2D-Vanilla-CNN, 80.8 seconds for the 2D-Hybrid-CNN. The number of training epochs is 20, total number of trials is 10, and the type of hardware used is a Linux server with Xeon 8176 CPU @2.1GHz, with 4 × 28 cores



## Training

In [ ]:
# Training the 3 models for 33 class
batch_size = 128
epochs = 1
seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores_1D_33 = []
cvscores_Vanilla_33 = []
cvscores_Hybrid_33 = []
times_1D = []
times_vanilla = []
times_hybrid = []
total_time = 0

start_time = time.time()
# model training loop: it is better to print the training/validation losses during the training
for j in range(1):
  i = 0
  for train, test in kfold.split(input_Xs_33, y_s_33):
    input_Xs = input_Xs_33.reshape(input_Xs_33.shape[0], img_rows_33, img_cols_33, 1)
    input_shape = (img_rows_33, img_cols_33, 1)
    input_Xs = input_Xs.astype('float32')

    # for the hybrid CNN 33 class, input cols and rows dimensions are flipped
    input_Xs_hybrid = input_Xs_33.reshape(input_Xs_33.shape[0], img_rows_hyb_33, img_cols_hyb_33, 1)
    input_shape_hybrid = (img_cols_33, img_rows_33, 1)
    input_img = Input(input_shape_hybrid)

    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(y_s_33)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    num_classes = len(onehot_encoded[0])

    # initialize models
    OneD_CNN = One_CNN(input_shape, num_classes_33)
    Vanilla_2D_CNN = Vanilla_33(input_shape, num_classes_33)
    Hybrid_CNN = Hybrid(input_img, num_classes_33)
    callbacks = [EarlyStopping(monitor='categorical_accuracy', patience=3, verbose=0)]

    if i==0:
      OneD_CNN.model.summary()
      Vanilla_2D_CNN.model.summary()
      Hybrid_CNN.model.summary()
      i = i + 1

    start_time_1D = time.time()
    # Train 1D CNN 33 class
    history_OneD = OneD_CNN.model.fit(input_Xs[train], onehot_encoded[train],
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0, callbacks=callbacks, validation_data=(input_Xs[test], onehot_encoded[test]))
    scores_oneD = OneD_CNN.model.evaluate(input_Xs[test], onehot_encoded[test], verbose=0)
    print("1D CNN 33 Class %s: %.2f%%" % ( OneD_CNN.model.metrics_names[1], scores_oneD[1]*100))
    cvscores_1D_33.append(scores_oneD[1] * 100)
    times_1D.append(time.time() - start_time_1D)

    start_time_vanilla = time.time()
    # Train 2D Vanilla CNN 33 class
    history_Vanilla = Vanilla_2D_CNN.model.fit(input_Xs[train], onehot_encoded[train],
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0, callbacks=callbacks, validation_data=(input_Xs[test], onehot_encoded[test]))
    scores_vanilla = Vanilla_2D_CNN.model.evaluate(input_Xs[test], onehot_encoded[test], verbose=0)
    print("Vanilla 2d CNN 33 Class %s: %.2f%%" % (Vanilla_2D_CNN.model.metrics_names[1], scores_vanilla[1]*100))
    cvscores_Vanilla_33.append(scores_vanilla[1] * 100)
    times_vanilla.append(time.time() - start_time_vanilla)

    start_time_hybrid = time.time()
    # Train Hybrid CNN 33 Class
    history_Hybrid = Hybrid_CNN.model.fit(input_Xs_hybrid[train], onehot_encoded[train],
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0, callbacks=callbacks, validation_data=(input_Xs_hybrid[test], onehot_encoded[test]))
    scores_hybrid = Hybrid_CNN.model.evaluate(input_Xs_hybrid[test], onehot_encoded[test], verbose=0)
    print("Hybrid CNN 33 Class %s: %.2f%%" % (Hybrid_CNN.model.metrics_names[1], scores_hybrid[1]*100))
    cvscores_Hybrid_33.append(scores_hybrid[1] * 100)
    times_hybrid.append(time.time() - start_time_hybrid)

avg_1D_time = sum(times_1D) / len(times_1D)
avg_vanilla_time = sum(times_vanilla) / len(times_vanilla)
avg_hybrid_time = sum(times_hybrid) / len(times_hybrid)
total_time = time.time() - start_time
print("avg 1D 33 class training time: %d" % avg_1D_time)
print("avg Vanilla 33 class training time: %d" % avg_vanilla_time)
print("avg Hybrid 33 class training time: %d" % avg_hybrid_time)
print("total training time: %d" % total_time)


In [ ]:
# Training the 3 models for 34 class
batch_size = 128
epochs = 1
seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

cvscores_1D = []
cv_yscores_1D = []
Y_test_1D = []

cvscores_vanilla = []
cv_yscores_vanilla = []
Y_test_vanilla = []

cvscores_hybrid = []
cv_yscores_hybrid = []
Y_test_hybrid = []

times_1D_34 = []
times_vanilla_34 = []
times_hybrid_34 = []
total_time_34 = 0

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_s_34)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

i = 0
start_time = time.time()
for train, test in kfold.split(X_cancer_samples_34, y_s_34):   # input_Xs in normal case and shuffled should be shuffled_Xs
  input_Xs = input_Xs_34.reshape(input_Xs_34.shape[0], img_rows_34, img_cols_34, 1)
  input_shape = (img_rows_34, img_cols_34, 1)
  input_Xs = input_Xs.astype('float32')

  # for the hybrid model
  input_img = Input(input_shape)

  num_classes = len(onehot_encoded[0])

  # initialize models
  OneD_CNN_34 = One_CNN(input_shape, num_classes)
  Vanilla_2D_CNN_34 = Vanilla_34(input_shape, num_classes)
  Hybrid_CNN_34 = Hybrid(input_img, num_classes)
  callbacks = [EarlyStopping(monitor='categorical_accuracy', patience=3, verbose=0)]
  if i==0:
    OneD_CNN_34.model.summary()
    Vanilla_2D_CNN_34.model.summary()
    Hybrid_CNN_34.model.summary()
    i = i + 1

  start_time_1D = time.time()
  # Train 1D CNN 34 class
  history_OneD = OneD_CNN_34.model.fit(input_Xs[train], onehot_encoded[train],
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=0, callbacks=callbacks, validation_data=(input_Xs[test], onehot_encoded[test]))
  scores_oneD = OneD_CNN_34.model.evaluate(input_Xs[test], onehot_encoded[test], verbose=0)
  y_score_1D = OneD_CNN_34.model.predict(input_Xs[test])
  print("1D CNN 34 Class %s: %.2f%%" % ( OneD_CNN_34.model.metrics_names[1], scores_oneD[1]*100))
  times_1D_34.append(time.time() - start_time_1D)

  cvscores_1D.append(scores_oneD[1] * 100)
  #Y_test_1D.append(onehot_encoded[test])
  #cv_yscores_1D.append(y_score_1D)

  start_time_vanilla = time.time()
  # Train 2D Vanilla CNN 34 class
  history_Vanilla = Vanilla_2D_CNN_34.model.fit(input_Xs[train], onehot_encoded[train],
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=0, callbacks=callbacks, validation_data=(input_Xs[test], onehot_encoded[test]))
  scores_vanilla_34 = Vanilla_2D_CNN_34 .model.evaluate(input_Xs[test], onehot_encoded[test], verbose=0)
  y_score_vanilla = Vanilla_2D_CNN_34 .model.predict(input_Xs[test])
  print("Vanilla_2D_CNN_34  %s: %.2f%%" % ( Vanilla_2D_CNN_34 .model.metrics_names[1], scores_vanilla_34[1]*100))
  times_vanilla_34.append(time.time() - start_time_vanilla)

  cvscores_vanilla.append(scores_vanilla_34[1] * 100)
  #Y_test_vanilla.append(onehot_encoded[test])
  #cv_yscores_vanilla.append(y_score_vanilla)

  start_time_hybrid = time.time()
  # Train Hybrid CNN 34 Class
  history_Hybrid = Hybrid_CNN_34.model.fit(input_Xs[train], onehot_encoded[train],
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=0, callbacks=callbacks, validation_data=(input_Xs[test], onehot_encoded[test]))
  scores_hybrid = Hybrid_CNN_34.model.evaluate(input_Xs[test], onehot_encoded[test], verbose=0)
  print("Hybrid CNN 34 Class %s: %.2f%%" % (Hybrid_CNN_34.model.metrics_names[1], scores_hybrid[1]*100))
  times_hybrid_34.append(time.time() - start_time_hybrid)

  cvscores_hybrid.append(scores_hybrid[1] * 100)

avg_1D_time_34 = sum(times_1D_34) / len(times_1D_34)
avg_vanilla_time_34 = sum(times_vanilla_34) / len(times_vanilla_34)
avg_hybrid_time_34 = sum(times_hybrid_34) / len(times_hybrid_34)
total_time_34 = time.time() - start_time
print("avg 1D 34 class training time: %d" % avg_1D_time_34)
print("avg Vanilla 34 class training time: %d" % avg_vanilla_time_34)
print("avg Hybrid 34 class training time: %d" % avg_hybrid_time_34)
print("total training time: %d" % total_time_34)


# Evaluation and Results
The metrics the original paper used to evaluate the models were accuracy and training loss. Since, I only used 1 epoch, it did not make sense to keep track of loss over epochs. So, in my reproduction, I only used accuracy to evaluate the models.


In [ ]:
# metrics to evaluate my model
cvscores_1D_33_mean = np.mean(cvscores_1D_33)
cvscores_vanilla_33_mean = np.mean(cvscores_Vanilla_33)
cvscores_hybrid_33_mean = np.mean(cvscores_Hybrid_33)
print("1D CNN 33 class Mean Categorical Accuracy: %.2f%% (+/- %.2f%%)" % (cvscores_1D_33_mean, np.std(cvscores_1D_33)))
print("Vanilla 2D CNN 33 class Mean  Categorical Accuracy: %.2f%% (+/- %.2f%%)" % (cvscores_vanilla_33_mean, np.std(cvscores_Vanilla_33)))
print("Hybrid CNN 33 class Mean Categorical Accuracy: %.2f%% (+/- %.2f%%)" % (cvscores_hybrid_33_mean, np.std(cvscores_Hybrid_33)))

cvscores_1D_34_mean = np.mean(cvscores_1D)
cvscores_vanilla_34_mean = np.mean(cvscores_vanilla)
cvscores_hybrid_34_mean = np.mean(cvscores_hybrid)
print("1D CNN 34 Class mean Categorical Accuracy: %.2f%% (+/- %.2f%%)" % (cvscores_1D_34_mean, np.std(cvscores_1D)))
print("Vanilla 2D CNN 34 Class mean Categorical Accuracy: %.2f%% (+/- %.2f%%)" % (cvscores_vanilla_34_mean, np.std(cvscores_vanilla)))
print("Hybrid CNN 34 Class mean Categorical Accuracy: %.2f%% (+/- %.2f%%)" % (cvscores_hybrid_34_mean, np.std(cvscores_hybrid)))

# plot to compare categorical accuracy
# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(12, 8))

# heights of bars
class_33 = [cvscores_1D_33_mean, cvscores_vanilla_33_mean, cvscores_hybrid_33_mean]
class_34 = [cvscores_1D_34_mean, cvscores_vanilla_34_mean, cvscores_hybrid_34_mean]

# Set position of bar on X axis
br1 = np.arange(len(class_33))
br2 = [x + barWidth for x in br1]

# Make the plot
plt.bar(br1, class_33, color ='b', width = barWidth,
        edgecolor ='grey', label ='33 Cancer Types')
plt.bar(br2, class_34, color ='y', width = barWidth,
        edgecolor ='grey', label ='33 Cancer Types + Normal')

# Adding Xticks
plt.title('Model Accuracies for 33 and 34 Class')
plt.ylabel('Mean Accuracy', fontweight ='bold', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(class_33))],
        ['1D-CNN', '2D-Vanilla-CNN', '2D-Hybrid-CNN'])
plt.ylim(ymin=70)

plt.legend()
plt.show()
#plt.savefig('Accuracy_Results.png')

# it is better to save the numbers and figures for your presentation.

## Model comparison (Results and Analysis and Ablation Study)

The original paper also trained a 2D-3Layer CNN as a baseline to compare the three models created in this paper against, but this baseline model was not included in the codebase. The 2D-3Layer CNN has significantly more parameters to train (the paper reported 26,211,233 parameters), resulting in a much longer time to train the model compared to the three models proposed in this paper.

Since I only ran the three proposed model in the paper with 1 epoch, my performance metrics will not compare to those in the paper. But, I can still compare the three models' metrics I reproduced against each other.

When trained with only the 33 cancer types, the 1D-CNN slightly outperformed the other two models with a mean accuracy of 90.51% compared to 89.1% and 89.87% of the Vanilla-CNN and the Hybrid-CNN respectively. When trained with the 33 cancer types plus the normal, the Hybrid-CNN outperformed the other two models with a mean accuracy of 89.43% compared to 86.79% and 87.31% of the 1D-CNN and Vanilla-CNN respectively. These number confirm that hypothesis 1 is true, that the three models proposed in the paper can achieve sufficient and comparable accuracy numbers to more complicated models.

Since, these number are all relatively close to each other, the simpler model with less parameters to train would be preferred. This would be the 1D-CNN or the Hybrid-CNN. The Hybrid-CNN performs better than the 1D-CNN in the 33 class plus normal dataset and performs similarly to the 1D-CNN in the 33 class dataset, but the Hybrid-CNN also has more parameters that need to be trained than the 1D-CNN. Also, in the original paper, the performance of the 1D-CNN was a lot closer to the Hybrid-CNN in the 33 class plus normal case, so it makes sense why the paper preferred the 1D-CNN as well.

Also when the normal samples were introduced, thus introducing the impact of tissue of origin to the models, the model accuracies were all lower compared to using only the 33 cancer types. But, the accuracies numbers were not significantly lower, the largest discrepancy in my reproduction being the 1D-CNN performance of 86.79% accuracy with normal samples compared to 90.51% without normal samples. This difference of 3.72% accuracy is not a large difference, so that means these three models are robust enough to handle the impact of tissue of origin. These results conclude that hypothesis 2 is true, these three models are robust enough to be able to take tissue of origin into account when predicting cancer types. Also, consider that in the original paper, with more epochs, the accuracy difference between the 33 class and the 33 plus normal class is even smaller.

My ablation study that I propsed was to compare the three simpler models proposed in the paper with a more complicated 2D-3Layer CNN. In my reproduction, the three simple models had accuracy metrics above 90% even when I only used 1 epoch. The paper's github did not contain code for the 2D-3Layer CNN, so I was not able to reproduce the 2D-3Layer model and the accuracy metrics for that model. But by going off the numbers in the original paper for the 2D-3Layer model they implemented, I am still able to compare my results in my reproduction (>90% accuracy) to the accuracy of the 2D-3Layer model results in the original paper (~95%), which shows that the three simple models that I trained are only 5% lower in accuracy to the 2D-3Layer model (also note that I trained with epoch of 1, so the actual gap should be even smaller, which is shown in the original paper).


# Discussion

The Paper is reproducible, but the model training takes too much time, which is why I reduced the epochs to 1, so I was not able to replicate the exact accuracy numbers that the original paper reported.

I found that getting the paper's code to run was easy, there were little bugs, just a couple of typos that I found. The only issue was with the hyperparameter training, the original paper used KerasClassifier, which is no longer in the sklearn package. I had to update to the latest sklearn package and use the scikeras package to be able to use KerasClassifier. But when I updated to the latest sklearn package, that broke the preprocessing code that used OneHotEncoder. So, I decided to not try to replicate the results of the hyperparameter training, but the hyperparameter training would take too long anyways, so there is not a lot to be gained by trying to repliate that in my paper. The main goal of the original paper was to evaluate the three models, and I was able to do that by using the optimal hyperparamters given by the original paper.

I found that organizing the paper's code was difficult. The paper has separate files for training each model, but there is a lot of duplicate code in each file. So it was difficult to condense the codebase into more structured and less duplicated code. For example using classes for the models and functions for getting the data. I found there were slight differences in the processing of the data in the Hybrid-CNN 33 class code compared to the 1D-CNN and Vanilla-CNN 33 class code. So I had to make slight adjustments in my code to account for that.

I would suggest to the author and other reproducers to make the codebase more readable and organized by introducing classes and functions. This will also remove replicated code in each of the files in the original codebase.

In the next phase or future, I would try to first replicate the original paper's accuracy metrics by using the original epoch numbers that the original paper used. Also, I may try using these 3 models with other data from The Cancer Genome Atlas to see how robust the models are.

# References

1.   Mostavi, M., Chiu, YC., Huang, Y. et al. Convolutional neural network models for cancer type prediction based on gene expression. BMC Med Genomics 13 (Suppl 5), 44 (2020). https://doi.org/10.1186/s12920-020-0677-2
2. Mostavi, M., CNNCancerType, (2020), GitHub repository, https://github.com/MMostavi/CNNCancerType

